## IMDB 영화평 감성분석(이진분류)

In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('./data/labeledTrainData.tsv', sep='\t', quoting=3)
df.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         25000 non-null  object
 1   sentiment  25000 non-null  int64 
 2   review     25000 non-null  object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


In [5]:
df.review[0][:1000]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

- 텍스트 전처리

In [6]:
# <br /> 공백 전환
df.review = df.review.str.replace('<br />',' ')

In [7]:
# 구둣점, 숫자 제거 - 영문자가 아닌 글자는 공백으로 전환
df.review = df.review.str.replace('[^A-Za-z]',' ').str.strip()
df.review[0][:1000]

C:\Users\lucky\AppData\Local\Temp\ipykernel_11692\1793999159.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df.review = df.review.str.replace('[^A-Za-z]',' ').str.strip()


'With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay   Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him   The actual feature film bit when it finally starts is only on for  

- Train / Test spilt

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df.review, df.sentiment, stratify=df.sentiment, random_state=2022
)
y_train.value_counts()

0    9375
1    9375
Name: sentiment, dtype: int64

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
cvect = CountVectorizer(stop_words='english')

In [15]:
cvect.fit(X_train)
X_train_cv = cvect.transform(X_train)
X_train_cv.shape

(18750, 65213)

In [17]:
# Test dataset 은 train dataset을 변환한 Vectorizer로 변환
X_test_cv = cvect.transform(X_test)
X_test_cv.shape

(6250, 65213)

- Classifier : Navie Bayes

In [19]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [21]:
%time nb.fit(X_train_cv, y_train)

CPU times: total: 0 ns
Wall time: 11 ms


MultinomialNB()

In [22]:
nb.score(X_test_cv, y_test)

0.85504

- ngram_range = (1, 2)

In [24]:
cvect_2 = CountVectorizer(ngram_range=(1, 2), stop_words='english')
cvect_2.fit(X_train)
%time X_train_cv2 = cvect_2.transform(X_train)
print(X_train_cv2.shape)

CPU times: total: 4.25 s
Wall time: 4.25 s
(18750, 1386558)


In [25]:
X_test_cv2 = cvect_2.transform(X_test)
X_test_cv2.shape

(6250, 1386558)

In [26]:
nb_2 = MultinomialNB()
nb_2.fit(X_train_cv2, y_train)

MultinomialNB()

In [27]:
nb_2.score(X_test_cv2, y_test)

0.8672

- model save & load

In [29]:
import joblib
joblib.dump(cvect_2, 'imdb_cvect_2.pkl')
joblib.dump(nb_2, 'imdb_nb_2.pkl')

['imdb_nb_2.pkl']

In [31]:
new_cvect = joblib.load('imdb_cvect_2.pkl')
new_nb = joblib.load('imdb_nb_2.pkl')

- 실제 데이터로 검증

In [33]:
review = '''
Robert P is a terrible Batman. Absolutely no Bat-charisma at all. 
The Riddler character is a complete bore! I hated all of this film. 
I certainly do not understand the good reviews. 
It's not close to the Dark Knight caliber. Please do not make a sequel.
'''

In [34]:
import re
review = re.sub('[^A-Za-z]', ' ', review).strip()

In [35]:
review_cv = new_cvect.transform([review])
review_cv.shape

(1, 1386558)

In [36]:
new_nb.predict(review_cv)

array([0], dtype=int64)